# WARC Index Status

We can use the Tracking Database check the WARC index status.

The CDX index status can be broken down by day using the following facet query (taking advantage of [Solr's JSON Facet API](https://lucene.apache.org/solr/guide/8_4/json-facet-api.html)).

In [3]:
import pandas as pd
import json
import requests

headers = {'content-type': "application/json" }

json_facet = {
    # Primary facet is by date - here we break down the last month(s) into days
    'facet': {
        'dates' : { 
            'type' : 'range', 
            'field' : 'timestamp_dt', 
            'start' : "NOW/DAY-6MONTH",
            'end' : "NOW/DAY", 
            'gap' : "+1DAY", 
            # For each day, we facet based on the CDX Index field, and make sure items with no value get recorded:
            'facet': { 
                'index_status': { 
                    'terms': { 
                        "field": "cdx_index_ss", 
                        'missing': True 
                    }
                }
            }
        } 
    }
}


params = {
  'q': 'kind_s:"warcs"',
  'rows': 0
}

r = requests.post("http://solr8.api.wa.bl.uk/solr/tracking/select", params=params, data=json.dumps(json_facet), headers=headers)

if r.status_code != 200:
    print(r.text)

from solr.solr_facet_helper import flatten_solr_buckets

df = pd.DataFrame(flatten_solr_buckets(r.json()['facets']))
# Filter empty rows:
df=df[df['count'] != 0]

df

,dates,index_status,count
0,2020-07-27T00:00:00Z,data-heritrix,87
2,2020-07-28T00:00:00Z,data-heritrix,91
4,2020-07-29T00:00:00Z,data-heritrix,117
6,2020-07-30T00:00:00Z,data-heritrix,129
7,2020-07-30T00:00:00Z,missing,1
...,...,...,...
357,2021-01-22T00:00:00Z,data-heritrix,318
359,2021-01-23T00:00:00Z,data-heritrix,318
361,2021-01-24T00:00:00Z,data-heritrix,314
363,2021-01-25T00:00:00Z,data-heritrix,326


Which can be used to build a simple visualisation:

In [4]:
import altair as alt

alt.Chart(df).mark_bar(size=6).encode(
    x='dates:T',
    y='count',
    color='index_status',
    tooltip=[alt.Tooltip('dates:T', format='%A, %e %B %Y'),'index_status', 'count']
).properties(width=600).interactive()

alt.Chart(...)